# Imports

In [1]:
import json
import time
from collections import Counter
from itertools import chain
from pathlib import Path

import numpy as np
import pandas as pd
import re
import regex

from src.companies.processor import clean_company_type, normalize_company_name
from src.nif_validation.validation import (
    get_info_from_cif,
    get_nif_type,
    is_valid_nif,
    validate_nif,
)
from src.utils.utils import fill_to_length, merge_orig_dataframes
from src.utils.utils_parallelization import parallelize_function

# Aux functions
Functions necessary for processing the cells

In [2]:
def nif_from_name(name):
    """Searches whether the NIF is included in the name and separates it"""
    name_spl = np.array(name.split())
    valid = np.array([bool(validate_nif(s)) for s in name_spl])
    new_name = " ".join(name_spl[~valid])
    new_nif = Counter(name_spl[valid]).most_common()[0][0] if valid.any() else np.nan
    return new_name, new_nif

In [3]:
import contextlib

@contextlib.contextmanager
def log_time(task_name: str):
    """Context manager to log the execution time of a block of code."""
    t0 = time.time()
    yield
    t1 = time.time()
    print(f"{task_name} - {t1-t0}")

    
def execute_function(func, data, prefer=None, workers=-1, *args, **kwargs):
    """Wrapper function to decide whether to use parallel processing or not."""
    if not prefer:
        return data.apply(func, *args, **kwargs)
    else:
        return parallelize_function(
            func,
            data,
            workers=workers,
            prefer=prefer,
            show_progress=True,
            leave=True,
            position=0,
            *args,
            **kwargs,
        )


def clean_df(df: pd.DataFrame, prefer=None, workers=-1):
    # Remove unwanted whitespace
    with log_time("Removing unwanted whitespace"):
        df = df.applymap(
            lambda x: regex.sub(r"((?<=\w+\W)\s+)|(\s+(?=\W\w+))", "", x)
            if not pd.isna(x)
            else None
        )

    # Validate NIF
    with log_time("Validating NIF"):
        df["ID"] = execute_function(validate_nif, df["ID"], prefer, workers)

    # Clean company type
    with log_time("Cleaning company type"):
        name = [
            regex.sub(i, "", n) if not (pd.isna(n) or pd.isna(i)) else n
            for i, n in df[["ID", "Name"]].values
        ]
        df["Name"] = execute_function(
            clean_company_type, name, prefer, workers, remove_type=False
        )

    # Remove company type
    with log_time("Removing company type"):
        df["Name_proc"] = execute_function(
            clean_company_type, df["Name"], prefer, workers, remove_type=True
        )

    # Normalize company name
    with log_time("Normalizing company name"):
        df["Name_norm"] = execute_function(
            normalize_company_name, df["Name_proc"], prefer, workers
        )

    return df

# Merge data from different sources

In [4]:
df_companies = merge_orig_dataframes(dir_metadata=Path("../nuevas_descargas/parquet_antiguos/"))
#df_companies.to_parquet("data/companies.parquet")

#df_companies = pd.read_parquet("data/companies.parquet")

/export/usuarios_ml4ds/cggamella/NP-Company-Process/src/utils/utils.py:101: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_companies = df_companies.applymap(fill_na, fill=[None])
/export/usuarios_ml4ds/cggamella/NP-Company-Process/src/utils/utils.py:101: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_companies = df_companies.applymap(fill_na, fill=[None])
/export/usuarios_ml4ds/cggamella/NP-Company-Process/src/utils/utils.py:101: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_companies = df_companies.applymap(fill_na, fill=[None])
/export/usuarios_ml4ds/cggamella/NP-Company-Process/src/utils/utils.py:122: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_companies = df_companies.applymap(lambda x: [None] if x is np.nan else x)


In [5]:
df_companies

,SMEAwardedIndicator,ID,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,id_tender
identifier,,,,,,,,,,
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/499,[None],[b30437347],[nif],[None],[climayor s.l. b30437347],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/498,[None],[b60564309],[nif],[None],"[gometrics, s.l.]",[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/497,[None],[g57694549],[nif],[None],[associaciò alcem el c.i.n.e.],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/496,[None],[None],[None],[None],[None],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722_12.atom/495,[None],[b73326019],[nif],[None],[diseño y decoraciones j. peñalver s.l. b73326...,[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...,...,...
PlataformasAgregadasSinMenores_202301.zip/PlataformasAgregadasSinMenores.atom/4,[None],[None],[None],[None],[None],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
PlataformasAgregadasSinMenores_202301.zip/PlataformasAgregadasSinMenores.atom/3,[None],[None],[None],[None],[None],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...
PlataformasAgregadasSinMenores_202301.zip/PlataformasAgregadasSinMenores.atom/2,[None],[None],[None],[None],[None],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...


# Obtain individual companies

In [6]:
# Use only those where all dimensions match
# (e.g. same number of companies and companies ids) and drop NAs
df_companies = df_companies[
    df_companies[["ID", "Name"]]
    .applymap(lambda x: not pd.isna(x[0]))
    .apply(all, axis=1)
]
df_companies = df_companies[
    df_companies.applymap(lambda x: len(x) if x[0] else None).apply(
        lambda x: len(set([el for el in x if not pd.isnull(el)])) == 1,
        axis=1,
    )
]
companies_columns = list(df_companies.columns)
# Get number of companies by tender
df_companies["_len"] = df_companies["ID"].apply(len)

# Fill lists of None to have the same number of elements and explode later
companies = pd.DataFrame(
    df_companies.apply(
        lambda x: [fill_to_length(list(el), x[-1]) for el in x[:-1]], axis=1
    ).tolist(),
    columns=companies_columns,
)

# Split companies in rows
companies = companies.explode(companies_columns)
companies = companies.reset_index(drop=True)
display(companies.head())

/tmp/ipykernel_2631029/2921704781.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: not pd.isna(x[0]))
/tmp/ipykernel_2631029/2921704781.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_companies.applymap(lambda x: len(x) if x[0] else None).apply(
/tmp/ipykernel_2631029/2921704781.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: [fill_to_length(list(el), x[-1]) for el in x[:-1]], axis=1


,SMEAwardedIndicator,ID,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,id_tender
0,None,b30437347,nif,None,climayor s.l. b30437347,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
1,None,b60564309,nif,None,"gometrics, s.l.",None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
2,None,g57694549,nif,None,associaciò alcem el c.i.n.e.,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
3,None,b73326019,nif,None,diseño y decoraciones j. peñalver s.l. b73326019,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
4,None,b28954170,nif,None,"thermo fisher scientific, s.l.",None,None,None,None,https://contrataciondelestado.es/sindicacion/d...


### Foreign/European IDs

In [7]:
identificadores_pais = pd.read_csv("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Company-Process/src/nif_validation/data/identificador_paises.csv")

In [8]:
companies_foreign = companies[
     companies["ID"].apply(
         lambda x: x[:2] in identificadores_pais["identificador"].values
     )
 ]
companies_foreign["ID"].apply(lambda x: x[:2]).value_counts()

ID
de    5939
fr    2834
nl    2296
es    1264
pt    1121
be    1072
ie    1009
it     850
se     312
at     294
lu     253
cz     214
dk     189
pl     143
fi      87
lt      69
el      55
bg      51
lv      46
hu      42
si      38
hr      37
cy      29
ee      23
sk      22
ro      21
mt      16
xi       2
Name: count, dtype: int64

In [9]:
# # European companies that are from Spain
companies_foreign_es = companies_foreign[
     companies_foreign["ID"].apply(lambda x: x[:2] == "es")
]
companies_foreign_es[
     companies_foreign_es["ID"].apply(lambda x: validate_nif(x[2:])).isna()
]

,SMEAwardedIndicator,ID,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,id_tender
5709,None,esa4113203,otros,None,"ayesa advanced technologies, s.a.",None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
7343,None,es000a46041711,otros,None,becsa,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
9009,None,es000a46041711,otros,None,becsa,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
9062,None,es000a46076873,otros,None,durantia,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
9068,None,es000a16134629,otros,None,"alvaro villescusa, s.a.",None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
...,...,...,...,...,...,...,...,...,...,...
2400894,None,es000b48434096/es000b85235430,None,None,"nacil medica 4 group s.l.7 / linet iberia,s.l.",None,None,None,None,https://contrataciondelestado.es/sindicacion/l...
2400903,None,es000b80601982,None,None,"cardiva 2,s.l.",None,None,None,None,https://contrataciondelestado.es/sindicacion/l...
2426680,None,es000b82646142,None,None,"bemasce tecnica,sl.",None,None,None,None,https://contrataciondelestado.es/sindicacion/l...
2543660,None,esb6614930,None,None,"cardiosos global protection, s.l",None,None,None,None,https://contrataciondelestado.es/sindicacion/p...


In [10]:
# # Companies with foreign ID that have a valid NIF in Spain
companies_foreign_valid = companies_foreign.loc[
     companies_foreign["ID"].apply(lambda x: validate_nif(x[2:])).dropna().index
]
companies_foreign_valid

,SMEAwardedIndicator,ID,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,id_tender
21295,None,esn0071290a,otros,None,microsoft,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
23146,None,esq2820002j,otros,None,ciemat,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
31817,None,esb06195044,otros,None,borrego cintas e hijos. s.l.,None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
35890,None,esa28315539,otros,None,"bruker española, sa",None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
52084,None,esa08431090,otros,None,"gas natural servicios sdg, s.a.",None,None,None,None,https://contrataciondelestado.es/sindicacion/d...
...,...,...,...,...,...,...,...,...,...,...
2624980,None,deb11657483,None,None,gutehoffnungshütte radsatz gmbh,None,None,None,None,https://contrataciondelestado.es/sindicacion/p...
2643444,None,deb11657483,None,None,gutehoffnungshütte radsatz gmbh,None,None,None,None,https://contrataciondelestado.es/sindicacion/p...
2643824,None,atu68849124,None,None,quality and safety in imaging gmbh,None,None,None,None,https://contrataciondelestado.es/sindicacion/p...
2649553,None,esb31231319,None,None,"excavaciones fermin osés, s.l.",None,None,None,None,https://contrataciondelestado.es/sindicacion/p...


### Clean companies info

In [11]:
with log_time("Clean df"):
    companies_clean = clean_df(companies, prefer="processes", workers=-1)

/tmp/ipykernel_2631029/4170415781.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(


Removing unwanted whitespace - 61.130215883255005


  0%|          | 0/2680269 [00:00<?, ?it/s]

Validating NIF - 15.594133615493774


  0%|          | 0/2680269 [00:00<?, ?it/s]

Cleaning company type - 223.85455894470215


  0%|          | 0/2680269 [00:00<?, ?it/s]

Removing company type - 51.58287072181702


  0%|          | 0/2680269 [00:00<?, ?it/s]

Normalizing company name - 11.270267009735107
Clean df - 363.45612263679504


In [12]:
# Aggregate company info in lists
companies_clean["SMEAwardedIndicator"] = companies_clean["SMEAwardedIndicator"].apply(
    lambda x: None if not x else True if x == "true" else False
)
companies_clean = (
    companies_clean
    # companies[["ID", "Name", "Name_proc", "Name_norm"]]
    .groupby(["ID", "Name_norm"])
    .agg(list)
    .reset_index()
)
companies_clean["count"] = companies_clean["Name_proc"].apply(len)
companies_clean = companies_clean.reset_index()

In [13]:
companies_clean

,index,ID,Name_norm,SMEAwardedIndicator,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,id_tender,Name_proc,count
0,0,00001014w,elcorteingles,[False],[nif],[None],[el corte ingles s.a.],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[el corte ingles],1
1,1,00021492x,carmenbalgueriasjimenez,"[None, None]","[nif, nif]","[None, None]","[carmen balguerias jiménez, carmen balguerias ...","[None, None]","[None, None]","[None, None]","[None, None]",[https://contrataciondelestado.es/sindicacion/...,"[carmen balguerias jiménez, carmen balguerias ...",2
2,2,00035211k,palomasainzdelamazadelaserna,[None],[nif],[None],[paloma sáinz de la maza de la serna],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[paloma sáinz de la maza de la serna],1
3,3,00067665e,albertodelgadocebrian,[None],[nif],[None],[alberto delgado cebrián],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[alberto delgado cebrián],1
4,4,00072839k,fernandezabadvicente,"[None, None]","[nif, otros]","[None, None]","[fernandez abad vicente, fernandez abad,vicente]","[None, None]","[None, None]","[None, None]","[None, None]",[https://contrataciondelestado.es/sindicacion/...,"[fernandez abad vicente, fernandez abad,vicente]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437834,437834,z0101664d,damianezequielaramendi,[False],[nif],[None],[damian ezequiel aramendi],[None],[es],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[damian ezequiel aramendi],1
437835,437835,z0112175d,gheorghealinstanca,[False],[nif],[None],[gheorghe alin stanca],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[gheorghe alin stanca],1
437836,437836,z0145297b,gerardomartinez,[False],[nif],[None],[gerardo martinez],[None],[None],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[gerardo martinez],1
437837,437837,z0425299b,gabrielgardiol,[True],[otros],[None],[gabriel gardiol],[None],[es],[None],[None],[https://contrataciondelestado.es/sindicacion/...,[gabriel gardiol],1


#### Unique names and IDs

In [14]:
# Unique names and IDs
# These companies have always appeared with the same (id-name) association
cols_vals = [
    c for c in companies_clean.columns if c not in ["ID", "Name_norm", "count"]
]
unique_ID = ~companies_clean["ID"].duplicated(keep=False)
unique_NAME = ~companies_clean["Name_norm"].duplicated(keep=False)

# Unique by ID and name
unique = companies_clean[unique_ID & unique_NAME].copy()

# Non unique IDs
non_unique_ids = list(set(companies_clean["index"]) - set(unique["index"]))
non_unique = companies_clean[companies_clean["index"].isin(non_unique_ids)].copy()

unique["index"] = unique["index"].apply(lambda x: [x])
non_unique["index"] = non_unique["index"].apply(lambda x: [x])
print(unique.shape, non_unique.shape)

(232592, 14) (205247, 14)


#### Repeated IDs and Names

In [15]:
# Choose definitive values
def suggest_value(elements):
    """
    Select elements based on appearance.
    If same number of appearances, choose the longest.
    If shorter elements are not included in the 'main' one, return all.
    """
    cnt = Counter(elements)
    cnt.pop(None, None)
    cnt = cnt.most_common()
    if cnt:
        max_cnt = cnt[0][1]
        els = sorted([k for k, v in cnt if v == max_cnt], key=lambda x: (-len(x), x))
        # return els[0]
        base = els.pop(0)
        return [base]
        # if all(
        #     [all(t in base for t in regex.sub(r"\W", " ", el).split()) for el in els]
        # ):
        #     return [base]
        # return [base] + els
    else:
        # return None
        return [None]


# Repeated IDs
def unify_repeated_col(df: pd.DataFrame, rep_col: str, un_col: str):
    """
    Takes a dataframe with duplicated values in one column that should be unique (e.g. repeated IDs)
    and another column that should also be unique given the previous one (e.g. title)
    and unifies it so that it chooses the best option.

    Parameters
    ----------
    df: pd.DataFrame
    rep_col: str
        Name of column with repeated values that will be unified
    un_col: str
        Name of column with non unique values
    """
    # Non-unique columns
    cols_vals = [c for c in df.columns if c not in [rep_col, "count", "index"]]
    repeated_rows = df[rep_col].duplicated(keep=False)
    repeated = df[repeated_rows]

    # Count times the values appear
    repeated.loc[repeated.index, [un_col]] = (
        repeated.loc[repeated.index, un_col].apply(lambda x: [x])
        * repeated.loc[repeated.index, "count"]
    )
    # Group by repeated
    repeated = repeated.reset_index()
    repeated = repeated.groupby(rep_col).agg(
        {
            # "index": list,
            "index": sum,
            **{c: lambda x: list(chain.from_iterable(x)) for c in cols_vals},
            "count": sum,
        }
    )
    # Get the most common values for each column
    repeated.loc[repeated.index, un_col] = (
        repeated.loc[repeated.index, un_col].apply(suggest_value).values
    )
    repeated = repeated.reset_index()

    # Concatenate unique
    use_index = repeated.loc[repeated[un_col].apply(len) == 1, un_col].index
    repeated.loc[use_index, un_col] = repeated.loc[use_index, un_col].apply(
        lambda x: x[0]
    )
    unified = repeated.loc[use_index]

    return unified

In [16]:
# Obtain unique ID-name
unified_ID = unify_repeated_col(non_unique, "ID", "Name_norm")
# Update non_unique
non_unique_ids = list(
    set(chain.from_iterable(non_unique["index"]))
    - set(chain.from_iterable(unified_ID["index"]))
)
# non_unique = companies_clean.loc[non_unique_ids]
non_unique = companies_clean[companies_clean["index"].isin(non_unique_ids)]
non_unique["index"] = non_unique["index"].apply(lambda x: [x])

/tmp/ipykernel_2631029/4236461406.py:54: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  repeated = repeated.groupby(rep_col).agg(
/tmp/ipykernel_2631029/4236461406.py:54: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  repeated = repeated.groupby(rep_col).agg(
/tmp/ipykernel_2631029/659276242.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_unique["index"] = non_unique["inde

In [17]:
# Obtain unique name-ID
unified_NAME = unify_repeated_col(non_unique, "Name_norm", "ID")
# Update non_unique
non_unique_ids = list(
    set(chain.from_iterable(non_unique["index"]))
    - set(chain.from_iterable(unified_NAME["index"]))
)
# non_unique = companies_clean.loc[non_unique_ids]
non_unique = companies_clean[companies_clean["index"].isin(non_unique_ids)]
non_unique["index"] = non_unique["index"].apply(lambda x: [x])

/tmp/ipykernel_2631029/4236461406.py:54: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  repeated = repeated.groupby(rep_col).agg(
/tmp/ipykernel_2631029/4236461406.py:54: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  repeated = repeated.groupby(rep_col).agg(
/tmp/ipykernel_2631029/1135489243.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_unique["index"] = non_unique["ind

#### Merge Companies info

In [18]:
# Global
# Merge unique+unifiedID+unifiedName+nonUnique
merged_global = pd.concat([unique, unified_ID, unified_NAME, non_unique])
cols_vals = [
    c
    for c in merged_global.columns
    if c not in ["ID", "Name_norm", "count", "index", "id_tender"]
]
merged_global = merged_global.groupby(["ID", "Name_norm"]).agg(
    {
        # "index": lambda x: list(chain.from_iterable(x)),
        "index": sum,
        "id_tender": sum,
        **{c: lambda x: list(chain.from_iterable(x)) for c in cols_vals},
        "count": sum,
    }
)
merged_global = merged_global.reset_index()
print(len(merged_global))
display(merged_global.head())

/tmp/ipykernel_2631029/775600221.py:9: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  merged_global = merged_global.groupby(["ID", "Name_norm"]).agg(
/tmp/ipykernel_2631029/775600221.py:9: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  merged_global = merged_global.groupby(["ID", "Name_norm"]).agg(


306749


,ID,Name_norm,index,id_tender,SMEAwardedIndicator,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,count
0,00001014w,elcorteingles,[0],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[el corte ingles s.a.],[None],[None],[None],[None],[el corte ingles],1
1,00021492x,carmenbalgueriasjimenez,[1],[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[nif, nif]","[None, None]","[carmen balguerias jiménez, carmen balguerias ...","[None, None]","[None, None]","[None, None]","[None, None]","[carmen balguerias jiménez, carmen balguerias ...",2
2,00035211k,palomasainzdelamazadelaserna,[2],[https://contrataciondelestado.es/sindicacion/...,[None],[nif],[None],[paloma sáinz de la maza de la serna],[None],[None],[None],[None],[paloma sáinz de la maza de la serna],1
3,00067665e,albertodelgadocebrian,[3],[https://contrataciondelestado.es/sindicacion/...,[None],[nif],[None],[alberto delgado cebrián],[None],[None],[None],[None],[alberto delgado cebrián],1
4,00072839k,fernandezabadvicente,[4],[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[nif, otros]","[None, None]","[fernandez abad vicente, fernandez abad,vicente]","[None, None]","[None, None]","[None, None]","[None, None]","[fernandez abad vicente, fernandez abad,vicente]",2


#### Unify found names

In [19]:
# Get all names found in the tenders
merged_global["UsedNames"] = (merged_global["Name"] + merged_global["Name_proc"]).apply(
    lambda x: sorted(list(set(x)))
)

#### Propose a final name

In [20]:
# Initial computations
data = merged_global["Name_proc"]
# local_frequencies = data.apply(lambda x: dict(Counter(x)))
local_frequencies = data.apply(lambda x: {k: v / len(x) for k, v in Counter(x).items()})
global_frequencies = data.explode().value_counts().to_dict()
global_frequencies = pd.Series(global_frequencies)
merged_global["Name_proposed"] = local_frequencies.apply(
    lambda x: sorted(x.items(), key=lambda el: el[1], reverse=True)[0][0]
)

In [ ]:
# # Calculate weighted scores and suggest a name for each ID
# suggested_names = {}
# for id_, weights in local_frequencies.items():
#     max_score = float("-inf")
#     suggested_name = None

#     # Escale the frequencies in current item
#     _this_global_frequencies = {name: global_frequencies[name] for name in weights}
#     escale_global_frequencies = {
#         name: _this_global_frequencies[name] / sum(_this_global_frequencies.values())
#         for name in _this_global_frequencies
#     }
#     for name, local_weight in weights.items():
#         weighted_score = (local_weight) / (escale_global_frequencies[name])
#         if weighted_score > max_score:
#             max_score = weighted_score
#             suggested_name = name
#     suggested_names[id_] = suggested_name

# merged_global["Name_proposed2"] = suggested_names.values()

In [ ]:
# merged_global.loc[
#     merged_global["Name_proposed2"] != merged_global["Name_proposed"],
#     ["Name_proposed2", "Name_proposed"],
# ].head()

#### Check if company is SME

In [21]:
def isPYME(SMEIndicators):
    # Evaluate if is SME based on the SMEAwardedIndicator appearances
    # If True and False are present, return None
    # TODO: make a better decision
    sme_counts = Counter(SMEIndicators)
    if True in sme_counts and False in sme_counts:
        return None
    return sme_counts.most_common(1)[0][0]


merged_global["isPYME"] = merged_global["SMEAwardedIndicator"].apply(isPYME)

#### Check CityName and PostalZone

In [22]:
def get_city_name(CityName):
    # Evaluate the city name based on the CityName appearances
    # Get most common excluding None
    # TODO: make a better decision
    city_names = Counter(CityName)
    if None in city_names.keys():
        city_names.pop(None)
    if not len(city_names) == 1:
        return None
    return city_names.most_common(1)[0][0]


def get_postal_zone(PostalZone):
    # Evaluate the postal zone based on the PostalZone appearances
    # Get most common excluding None
    # TODO: make a better decision
    postal_zones = Counter(PostalZone)
    if None in postal_zones.keys():
        postal_zones.pop(None)
    if not len(postal_zones) == 1:
        return None
    return postal_zones.most_common(1)[0][0].split(".")[0]


merged_global["City"] = merged_global["CityName"].apply(get_city_name)
merged_global["PostalCode"] = merged_global["PostalZone"].apply(get_postal_zone)

In [23]:
merged_global

,ID,Name_norm,index,id_tender,SMEAwardedIndicator,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,PostalZone,CountrySubentityCode,Name_proc,count,UsedNames,Name_proposed,isPYME,City,PostalCode
0,00001014w,elcorteingles,[0],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[el corte ingles s.a.],[None],[None],[None],[None],[el corte ingles],1,"[el corte ingles, el corte ingles s.a.]",el corte ingles,False,None,None
1,00021492x,carmenbalgueriasjimenez,[1],[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[nif, nif]","[None, None]","[carmen balguerias jiménez, carmen balguerias ...","[None, None]","[None, None]","[None, None]","[None, None]","[carmen balguerias jiménez, carmen balguerias ...",2,[carmen balguerias jiménez],carmen balguerias jiménez,None,None,None
2,00035211k,palomasainzdelamazadelaserna,[2],[https://contrataciondelestado.es/sindicacion/...,[None],[nif],[None],[paloma sáinz de la maza de la serna],[None],[None],[None],[None],[paloma sáinz de la maza de la serna],1,[paloma sáinz de la maza de la serna],paloma sáinz de la maza de la serna,None,None,None
3,00067665e,albertodelgadocebrian,[3],[https://contrataciondelestado.es/sindicacion/...,[None],[nif],[None],[alberto delgado cebrián],[None],[None],[None],[None],[alberto delgado cebrián],1,[alberto delgado cebrián],alberto delgado cebrián,None,None,None
4,00072839k,fernandezabadvicente,[4],[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[nif, otros]","[None, None]","[fernandez abad vicente, fernandez abad,vicente]","[None, None]","[None, None]","[None, None]","[None, None]","[fernandez abad vicente, fernandez abad,vicente]",2,"[fernandez abad vicente, fernandez abad,vicente]",fernandez abad vicente,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306744,z0101664d,damianezequielaramendi,[437834],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[damian ezequiel aramendi],[None],[es],[None],[None],[damian ezequiel aramendi],1,[damian ezequiel aramendi],damian ezequiel aramendi,False,None,None
306745,z0112175d,gheorghealinstanca,[437835],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[gheorghe alin stanca],[None],[None],[None],[None],[gheorghe alin stanca],1,[gheorghe alin stanca],gheorghe alin stanca,False,None,None
306746,z0145297b,gerardomartinez,[437836],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[gerardo martinez],[None],[None],[None],[None],[gerardo martinez],1,[gerardo martinez],gerardo martinez,False,None,None
306747,z0425299b,gabrielgardiol,[437837],[https://contrataciondelestado.es/sindicacion/...,[True],[otros],[None],[gabriel gardiol],[None],[es],[None],[None],[gabriel gardiol],1,[gabriel gardiol],gabriel gardiol,True,None,None


### Add info

In [24]:
# Add information based on NIF
merged_global["NIF_type"] = merged_global["ID"].apply(get_nif_type)
merged_global["prov"], merged_global["comp_type"], merged_global["comp_desc"] = list(
    zip(*merged_global["ID"].apply(get_info_from_cif))
)
merged_global["comp_type"] = merged_global["comp_type"].apply(
    lambda x: x.split(",")[0] if not pd.isna(x) else None
)

### Find UTEs

In [25]:
values = merged_global["ID"].str.startswith("u")

# Filtrar y seleccionar los valores de la columna 'ID' que cumplen con la condición
ids_que_cumplen = merged_global.loc[values, "ID"]

# Filtrar y seleccionar las filas completas donde los valores de la columna 'ID' comienzan con "u"
filas_que_cumplen = merged_global.loc[values]

# Imprimir las filas completas que cumplen con la condición
print(filas_que_cumplen)

               ID                                          Name_norm  \
296766  u01523190                                   alegriasarasola2   
296767  u01548726                indenortpvproviseribericaieslizardi   
296768  u01561570                                 tranviauniversidad   
296769  u01563154                                    ceipkarmengoama   
296770  u01563196                                     indralkskz2018   
...           ...                                                ...   
302106  u99558694  indutecinatalacionesmarcoinfraestructurasymedi...   
302107  u99563744  indutecinstalacionesampersistemasmarcoinfrayme...   
302108  u99563751                    insaemainsaagromercadolosalamos   
302109  u99568222                   transfersmvsantiagoanguloaltemir   
302110  u99569147      insaeinfraestructurasmarcoinfraestructurasyma   

           index                                          id_tender  \
296766  [425032]  [https://contrataciondelestado.es/sindicacion/

In [26]:
merged_global['FullName'] = merged_global['UsedNames'].apply(lambda x: max(x, key=len))

In [27]:
merged_global

,ID,Name_norm,index,id_tender,SMEAwardedIndicator,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,...,UsedNames,Name_proposed,isPYME,City,PostalCode,NIF_type,prov,comp_type,comp_desc,FullName
0,00001014w,elcorteingles,[0],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[el corte ingles s.a.],[None],[None],...,"[el corte ingles, el corte ingles s.a.]",el corte ingles,False,None,None,DNI,None,None,None,el corte ingles s.a.
1,00021492x,carmenbalgueriasjimenez,[1],[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[nif, nif]","[None, None]","[carmen balguerias jiménez, carmen balguerias ...","[None, None]","[None, None]",...,[carmen balguerias jiménez],carmen balguerias jiménez,None,None,None,DNI,None,None,None,carmen balguerias jiménez
2,00035211k,palomasainzdelamazadelaserna,[2],[https://contrataciondelestado.es/sindicacion/...,[None],[nif],[None],[paloma sáinz de la maza de la serna],[None],[None],...,[paloma sáinz de la maza de la serna],paloma sáinz de la maza de la serna,None,None,None,DNI,None,None,None,paloma sáinz de la maza de la serna
3,00067665e,albertodelgadocebrian,[3],[https://contrataciondelestado.es/sindicacion/...,[None],[nif],[None],[alberto delgado cebrián],[None],[None],...,[alberto delgado cebrián],alberto delgado cebrián,None,None,None,DNI,None,None,None,alberto delgado cebrián
4,00072839k,fernandezabadvicente,[4],[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[nif, otros]","[None, None]","[fernandez abad vicente, fernandez abad,vicente]","[None, None]","[None, None]",...,"[fernandez abad vicente, fernandez abad,vicente]",fernandez abad vicente,None,None,None,DNI,None,None,None,fernandez abad vicente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306744,z0101664d,damianezequielaramendi,[437834],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[damian ezequiel aramendi],[None],[es],...,[damian ezequiel aramendi],damian ezequiel aramendi,False,None,None,NIE,None,None,None,damian ezequiel aramendi
306745,z0112175d,gheorghealinstanca,[437835],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[gheorghe alin stanca],[None],[None],...,[gheorghe alin stanca],gheorghe alin stanca,False,None,None,NIE,None,None,None,gheorghe alin stanca
306746,z0145297b,gerardomartinez,[437836],[https://contrataciondelestado.es/sindicacion/...,[False],[nif],[None],[gerardo martinez],[None],[None],...,[gerardo martinez],gerardo martinez,False,None,None,NIE,None,None,None,gerardo martinez
306747,z0425299b,gabrielgardiol,[437837],[https://contrataciondelestado.es/sindicacion/...,[True],[otros],[None],[gabriel gardiol],[None],[es],...,[gabriel gardiol],gabriel gardiol,True,None,None,NIE,None,None,None,gabriel gardiol


In [28]:
# Ampliar la expresión regular precompilada para capturar "UTE" con variaciones, "union temporal empresas", y sus siglas
pattern = re.compile(r"\b(u(\.)?t(\.)?e|union temporal empresas|uniones temporales de empresas)\b", re.IGNORECASE)

# Búsqueda de UTEs basada en nombres en la columna 'UsedNames'
ute_n = merged_global["UsedNames"].apply(lambda x: bool(pattern.search(" ".join([word.lower() for word in x]))))

# Búsqueda de UTEs basada en ID
ute_i = merged_global["ID"].str.startswith("u")

# Aplicar filtros basados en las columnas 'comp_type' y 'comp_desc' usando la expresión regular
ute_c_type = merged_global["comp_type"].apply(lambda x: bool(pattern.search(x.lower()) if pd.notnull(x) else False))
ute_c_desc = merged_global["comp_desc"].apply(lambda x: bool(pattern.search(x.lower()) if pd.notnull(x) else False))

# Combinar todos los filtros para encontrar UTEs basados en nombres, ID, comp_type, y comp_desc
utes_combined = merged_global[ute_n | ute_i | ute_c_type | ute_c_desc]

# Eliminar duplicados basándose en la columna 'ID', manteniendo la primera aparición
utes = utes_combined.drop_duplicates(subset="ID")

In [29]:
utes

,ID,Name_norm,index,id_tender,SMEAwardedIndicator,IDschemeName,CompanyTypeCode,Name,CityName,IdentificationCode,...,UsedNames,Name_proposed,isPYME,City,PostalCode,NIF_type,prov,comp_type,comp_desc,FullName
166,00400871g,ferrannavazoherrerofaranazoliverfortezaferre,"[206, 207]",[https://contrataciondelestado.es/sindicacion/...,"[True, None]","[None, None]","[None, None]","[u.t.e. ferranherrero f.aranaz, u.t.e. ferran ...","[None, None]","[es, None]",...,[ferran navazo herrero f.aranaz oliver forteza...,ferranherrero f.aranaz,True,None,None,DNI,None,None,None,u.t.e. ferran navazo herrero f.aranaz oliver f...
235,00416934j,getinsaverdascoarquitectos,[283],[https://contrataciondelestado.es/sindicacion/...,[False],[None],[None],[u.t.e. getinsa+verdasco arquitectos],[None],[es],...,"[getinsa+verdasco arquitectos, u.t.e. getinsa+...",getinsa+verdasco arquitectos,False,None,None,DNI,None,None,None,u.t.e. getinsa+verdasco arquitectos
2962,03429732h,munozmarineroluisjoaquin,"[3611, 3612, 3613]",[https://contrataciondelestado.es/sindicacion/...,"[None, None, None, None, None, None, True, Non...","[None, nif, nif, nif, nif, nif, nif, None, nif...","[None, None, None, None, None, None, None, Non...","[u.t.e. cuéllar, luis joaquin muñoz marinero, ...","[None, None, None, None, cuellar, None, None, ...","[None, None, None, None, es, None, es, es, Non...",...,"[cuéllar, luis joaquin muñoz marinero, muñoz m...","muñoz marinero,luis joaquin",None,None,44022,DNI,None,None,None,luis joaquin muñoz marinero
4846,04200682p,saramorenosoriayunomasunologicomcley1882numero1,[5847],[https://contrataciondelestado.es/sindicacion/...,[None],[None],[None],[sara moreno soria y uno mas uno logico mc u.t...,[None],[None],...,[sara moreno soria y uno mas uno logico mc ley...,sara moreno soria y uno mas uno logico mc ley ...,None,None,None,DNI,None,None,None,sara moreno soria y uno mas uno logico mc u.t....
12528,08762783j,julianprietojorgelopezjaviergomez,"[15716, 15717]",[https://contrataciondelestado.es/sindicacion/...,"[None, None]","[None, None]","[None, None]",[u.t.e. julian prieto - jorge lopez - javier g...,"[None, None]","[None, es]",...,"[julian prieto - jorge lopez - javier gomez, p...",julian prieto - jorge lopez - javier gomez,None,None,None,DNI,None,None,None,u.t.e. julian prieto - jorge lopez - javier gomez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302550,v67273987,girocopisistemesdorganitzacio,[432069],[https://contrataciondelestado.es/sindicacion/...,[None],[None],[None],[girocopi s.l. sistemes d organització s.a u.t...,[None],[None],...,[girocopi s.l. sistemes d organització s.a u.t...,girocopi sistemes d organització,None,None,None,CIF,Gerona,s.a.t.,Sociedad Agraria de Transformación,girocopi s.l. sistemes d organització s.a u.t.e.
302643,v87879375,aurenauditoresspred2redconsultores,[432216],[https://contrataciondelestado.es/sindicacion/...,[None],[None],[None],[u.t.e. auren auditores sp s.l.p. - red2red co...,[None],[None],...,"[auren auditores sp red2red consultores, u.t.e...",auren auditores sp red2red consultores,None,None,None,CIF,Madrid,s.a.t.,Sociedad Agraria de Transformación,u.t.e. auren auditores sp s.l.p. - red2red con...
302661,v88299789,ceviamepcycasasdelaalcarrialey181982de26demayo,[432237],[https://contrataciondelestado.es/sindicacion/...,[None],[None],[None],[ceviam epc s.l. y casas de la alcarria s.l u....,[None],[None],...,[ceviam epc s.l. y casas de la alcarria s.l u....,"ceviam epc y casas de la alcarria ,ley 18 1982...",None,None,None,CIF,Madrid,s.a.t.,Sociedad Agraria de Transformación,ceviam epc s.l. y casas de la alcarria s.l u.t...
302681,v90441049,caem,[432263],[https://contrataciondelestado.es/sindicacion/...,[None],[None],[None],[u.t.e. caem],[None],[None],...,"[caem, u.t.e. caem]",caem,None,None,None,CIF,Sevilla,s.a.t.,Sociedad Agraria de Transformación,u.t.e. caem


### Save data

In [30]:
provisional_utes_info = utes.rename(
    columns={
        "ID": "NIF",
        "id_tender": "id_tender",
        "Name_proposed": "Name2",
        "prov": "Province",
        "NIF_type": "NIFtype",
        "comp_type": "CompanyType",
        "comp_desc": "CompanyDescription",
    }
)[
    [
        "NIF",
        "FullName",
        "Name2",
        "Province",
        "NIFtype",
        "CompanyType",
        "CompanyDescription",
        "id_tender",
    ]
]

In [31]:
provisional_utes_info

,NIF,FullName,Name2,Province,NIFtype,CompanyType,CompanyDescription,id_tender
166,00400871g,u.t.e. ferran navazo herrero f.aranaz oliver f...,ferranherrero f.aranaz,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
235,00416934j,u.t.e. getinsa+verdasco arquitectos,getinsa+verdasco arquitectos,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
2962,03429732h,luis joaquin muñoz marinero,"muñoz marinero,luis joaquin",None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
4846,04200682p,sara moreno soria y uno mas uno logico mc u.t....,sara moreno soria y uno mas uno logico mc ley ...,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
12528,08762783j,u.t.e. julian prieto - jorge lopez - javier gomez,julian prieto - jorge lopez - javier gomez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...
302550,v67273987,girocopi s.l. sistemes d organització s.a u.t.e.,girocopi sistemes d organització,Gerona,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...
302643,v87879375,u.t.e. auren auditores sp s.l.p. - red2red con...,auren auditores sp red2red consultores,Madrid,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...
302661,v88299789,ceviam epc s.l. y casas de la alcarria s.l u.t...,"ceviam epc y casas de la alcarria ,ley 18 1982...",Madrid,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...
302681,v90441049,u.t.e. caem,caem,Sevilla,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...


In [32]:
# Ahora, renombrar la columna 'Name2' a 'Name'
provisional_utes_info = provisional_utes_info.rename(
    columns={
        "Name2": "Name"  
    }
)

In [33]:
provisional_utes_info

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender
166,00400871g,u.t.e. ferran navazo herrero f.aranaz oliver f...,ferranherrero f.aranaz,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
235,00416934j,u.t.e. getinsa+verdasco arquitectos,getinsa+verdasco arquitectos,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
2962,03429732h,luis joaquin muñoz marinero,"muñoz marinero,luis joaquin",None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
4846,04200682p,sara moreno soria y uno mas uno logico mc u.t....,sara moreno soria y uno mas uno logico mc ley ...,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
12528,08762783j,u.t.e. julian prieto - jorge lopez - javier gomez,julian prieto - jorge lopez - javier gomez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...
302550,v67273987,girocopi s.l. sistemes d organització s.a u.t.e.,girocopi sistemes d organització,Gerona,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...
302643,v87879375,u.t.e. auren auditores sp s.l.p. - red2red con...,auren auditores sp red2red consultores,Madrid,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...
302661,v88299789,ceviam epc s.l. y casas de la alcarria s.l u.t...,"ceviam epc y casas de la alcarria ,ley 18 1982...",Madrid,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...
302681,v90441049,u.t.e. caem,caem,Sevilla,CIF,s.a.t.,Sociedad Agraria de Transformación,[https://contrataciondelestado.es/sindicacion/...


In [ ]:
# Filtrar los valores no nulos de la columna 'CompanyDescription'
company_descriptions_not_null = provisional_utes_info[provisional_utes_info['CompanyDescription'].notnull()]

# Iterar sobre los valores no nulos e imprimirlos
for description in company_descriptions_not_null['CompanyDescription']:
    print(set(description))

In [39]:
provisional_company_info = merged_global.rename(
    columns={
        "ID": "NIF",
        "id_tender": "id_tender",
        "Name_proposed": "Name1",
        "prov": "Province",
        "NIF_type": "NIFtype",
        "comp_type": "CompanyType",
        "comp_desc": "CompanyDescription",
    }
)[
    [
        "NIF",
        "FullName",
        "Name1",
        "Province",
        "NIFtype",
        "CompanyType",
        "CompanyDescription",
        "id_tender",
    ]
]

In [40]:
provisional_company_info

,NIF,FullName,Name1,Province,NIFtype,CompanyType,CompanyDescription,id_tender
0,00001014w,el corte ingles s.a.,el corte ingles,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
1,00021492x,carmen balguerias jiménez,carmen balguerias jiménez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
2,00035211k,paloma sáinz de la maza de la serna,paloma sáinz de la maza de la serna,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
3,00067665e,alberto delgado cebrián,alberto delgado cebrián,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
4,00072839k,fernandez abad vicente,fernandez abad vicente,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...
306744,z0101664d,damian ezequiel aramendi,damian ezequiel aramendi,None,NIE,None,None,[https://contrataciondelestado.es/sindicacion/...
306745,z0112175d,gheorghe alin stanca,gheorghe alin stanca,None,NIE,None,None,[https://contrataciondelestado.es/sindicacion/...
306746,z0145297b,gerardo martinez,gerardo martinez,None,NIE,None,None,[https://contrataciondelestado.es/sindicacion/...
306747,z0425299b,gabriel gardiol,gabriel gardiol,None,NIE,None,None,[https://contrataciondelestado.es/sindicacion/...


In [41]:
# Ahora, renombrar la columna 'Name1' a 'Name'
provisional_company_info = provisional_company_info.rename(
    columns={
        "Name1": "Name"  # Renombrando Name1 a Name
    }
)

In [42]:
provisional_company_info.FullName.str.contains("xylem water solutions españa s.l.u.").notna()

0         True
1         True
2         True
3         True
4         True
          ... 
306744    True
306745    True
306746    True
306747    True
306748    True
Name: FullName, Length: 306749, dtype: bool

In [43]:
valores_no_nulos = provisional_company_info['CompanyType'].notna()

# Filtrar el DataFrame para obtener solo las filas con valores no nulos en la columna CompanyType
datos_no_nulos = provisional_company_info[valores_no_nulos]

# Convertir los valores no nulos en una lista si es necesario
valores_no_nulos_lista = datos_no_nulos['CompanyType'].tolist()

# Imprimir los valores no nulos
print(set(valores_no_nulos_lista))

{'s.r.c.', 'c.b.', 'coop.', 's.a.', 'u.t.e.', 's.c.', 's.a.t.', 's.l.', 's.e.n.c.'}


In [44]:
valores_que_comienzan_con_xylem = provisional_company_info['Name'].str.startswith("xylem")

# Filtra el DataFrame para mostrar solo los valores que no son False
valores_verdaderos = provisional_company_info[valores_que_comienzan_con_xylem]

# Imprime los valores que cumplen con la condición
print(valores_verdaderos.Name.iloc[0])

xylem water solutions españa


In [46]:
# provisional_company_info.to_parquet("data/provisional_company_info.parquet")
provisional_utes_info.to_parquet("data/utes_nuevo.parquet")

# provisional_company_info = pd.read_parquet("data/provisional_company_info.parquet")
# utes = pd.read_parquet("data/utes.parquet")

In [47]:
df_antiguo = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Company-Process/data/company_info.parquet")

In [48]:
df_antiguo

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender
0,00021492x,carmen balguerias jiménez,carmen balguerias jiménez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
1,00035211k,paloma sáinz de la maza de la serna,paloma sáinz de la maza de la serna,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
2,00067665e,alberto delgado cebrián,alberto delgado cebrián,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
3,00072839k,fernandez abad vicente,fernandez abad vicente,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
4,00076938a,luis terán lópez,luis terán lópez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...
267238,b28672764,xylem water solutions españa s.l.u.,xylem water solutions españa,Madrid,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...
267239,b99289795,zaragoza y eventos s.l.,zaragoza y eventos,Zaragoza,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...
267240,a28011153,zardoya otis s.a.,zardoya otis,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...
267241,b45273406,zenith toledo,zenith toledo,Toledo,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...


In [51]:
provisional_company_info.id_tender.iloc[0]

['https://contrataciondelestado.es/sindicacion/datosabiertosmenores/12263073']

In [50]:
# Hacemos un merge entre los dos dataframes especificando un outer join y el indicador
merged_df = pd.merge(provisional_company_info, df_antiguo, on='NIF', how='outer', indicator=True)

# Filtro para obtener solo las filas que están en 'df_nuevo_company' pero no en 'df_antiguo_company'
filas_unicas_en_nuevo = merged_df[merged_df['_merge'] == 'left_only']
filas_unicas_en_nuevo = filas_unicas_en_nuevo.drop(columns=['_merge'])

print(filas_unicas_en_nuevo)

              NIF                       FullName_x  \
0       00001014w             el corte ingles s.a.   
8       00110603l             baron crespo enrique   
12      00115258m  francisco javier vellés montoya   
13      00116517e    joaquin cotroneo diaz del rio   
26      00136242j                audello tetro sas   
...           ...                              ...   
306885  z0043657p              paul gilles massard   
306887  z0101664d         damian ezequiel aramendi   
306888  z0112175d             gheorghe alin stanca   
306890  z0425299b                  gabriel gardiol   
306891  z0482150y            angela clara pusceddu   

                                 Name_x Province_x NIFtype_x CompanyType_x  \
0                       el corte ingles       None       DNI          None   
8                  baron crespo enrique       None       DNI          None   
12      francisco javier vellés montoya       None       DNI          None   
13        joaquin cotroneo diaz del rio

In [150]:
data2 = df_antiguo['Name'].str.contains("zesauro traducciones")
provisional_company_info[data]

,NIF,NameNew,UsedNames,Province,NIFtype,CompanyType,CompanyDescription,id_tender
233274,b83230870,zesauro traducciones,"[zesauro traducciones, zesauro traducciones s.l.]",Madrid,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...


In [90]:
# Filtra directamente las filas donde 'CompanyDescription' comienza con "Sociedades de"
valores_que_comienzan_con_xylem = df_antiguo['CompanyDescription'].notna()

# Imprime los valores que cumplen con la condición
print(set(valores_que_comienzan_con_xylem))

{False, True}


In [153]:
df_antiguo

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender
0,00021492x,carmen balguerias jiménez,carmen balguerias jiménez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
1,00035211k,paloma sáinz de la maza de la serna,paloma sáinz de la maza de la serna,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
2,00067665e,alberto delgado cebrián,alberto delgado cebrián,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
3,00072839k,fernandez abad vicente,fernandez abad vicente,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
4,00076938a,luis terán lópez,luis terán lópez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...
267238,b28672764,xylem water solutions españa s.l.u.,xylem water solutions españa,Madrid,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...
267239,b99289795,zaragoza y eventos s.l.,zaragoza y eventos,Zaragoza,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...
267240,a28011153,zardoya otis s.a.,zardoya otis,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...
267241,b45273406,zenith toledo,zenith toledo,Toledo,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...


## Empresas Zaragoza

In [ ]:
# Cargar las empresas de Zaragoza y darle el formato propio
emp = pd.read_excel("data/licitador_09_23.xls")
emp_zgz = emp[["ID_EMPRESA", "NIF", "NOMBRE", "UTE"]].copy()
# Limpieza de nombres
emp_zgz["empresa"] = emp_zgz["NOMBRE"].apply(clean_company_type)
emp_zgz["empresa_proc"] = emp_zgz["empresa"].apply(clean_company_type, remove_type=True)
# Creamos una columna con los nombres que se han usado para identificar la empresa
emp_zgz["UsedNames"] = emp_zgz[["empresa", "empresa_proc"]].apply(
    lambda x: list(set([x[0], x[1]])), axis=1
)
emp_zgz["nif"] = emp_zgz["NIF"].apply(
    lambda x: regex.sub(r"\W", "", x.lower()) if not pd.isna(x) else None
)
emp_zgz["valid_nif"] = emp_zgz["nif"].apply(lambda x: is_valid_nif(x) if x else None)
emp_zgz["nif_type"] = emp_zgz["nif"].apply(lambda x: get_nif_type(x) if x else None)

# Transformar UTE a bool
emp_zgz["UTE"] = emp_zgz["UTE"].apply(
    lambda x: True if x == "S" else False if x == "N" else None
)

# Stats
print("Elementos inválidos en los datos:")
emp_zgz.apply(pd.isna, axis=1).sum()

### Empresas repetidas

In [ ]:
total_empresas = len(emp_zgz)

# Identificar NIF repetidos
emp_zgz_dup_nif = emp_zgz.loc[
    emp_zgz["nif"].duplicated(keep=False) & emp_zgz["nif"].notna(),
    ["ID_EMPRESA", "nif", "empresa_proc"],
].sort_values(by="nif")
print(f"Hay {len(emp_zgz_dup_nif)} NIFs duplicados:")
display(emp_zgz_dup_nif)
print(
    f"Los NIFs duplicados representan el {(len(emp_zgz_dup_nif) / total_empresas) * 100:.2f}% del total."
)

print()

# Identificar nombres repetidos
emp_zgz_dup_name = emp_zgz.loc[
    emp_zgz["empresa_proc"].duplicated(keep=False) & emp_zgz["empresa_proc"].notna(),
    ["ID_EMPRESA", "nif", "empresa_proc"],
].sort_values(by="empresa_proc")
print(f"Hay {len(emp_zgz_dup_name)} nombres duplicados:")
display(emp_zgz_dup_name)
print(
    f"Los nombres duplicados representan el {(len(emp_zgz_dup_name) / total_empresas) * 100:.2f}% del total."
)

### Search companies in own data

In [ ]:
own_info = provisional_company_info[
    ["NIF", "UsedNames", "NameProposed", "isPYME"]
].rename(columns={"NIF": "nif"})
own_info["UsedNames"] = own_info["UsedNames"].apply(list)

In [ ]:
# Buscar por NIF
nifs_usados = own_info.reset_index()
nif_zgz = emp_zgz[["ID_EMPRESA", "nif", "UTE", "empresa", "UsedNames"]]
found_by_nif = pd.merge(
    nif_zgz, nifs_usados, left_on="nif", right_on="nif", how="inner"
)
found_by_nif["UsedNames"] = found_by_nif[["UsedNames_x", "UsedNames_y"]].sum(axis=1)
found_by_nif = (
    found_by_nif.groupby("ID_EMPRESA")
    .agg(
        {
            "nif": set,
            "index": set,
            "UsedNames": sum,
            "NameProposed": set,
            "isPYME": set,
            "UTE": set,
        }
    )
    .applymap(lambda x: list(set(x)))
    .reset_index()
)
found_by_nif

In [ ]:
# found_by_name.loc[found_by_name["UsedNames"].apply(lambda x: "reby rides" in x)]

In [ ]:
# Buscar por nombre excluyendo las que hemos encontrado por NIF
nombres_usados = own_info.explode(column="UsedNames").reset_index()
nombres_zgz = emp_zgz[~emp_zgz["ID_EMPRESA"].isin(found_by_nif["ID_EMPRESA"])]
nombres_zgz = nombres_zgz[["ID_EMPRESA", "nif", "UTE", "empresa", "UsedNames"]].explode(
    column="UsedNames"
)
found_by_name = pd.merge(
    nombres_zgz, nombres_usados, left_on="UsedNames", right_on="UsedNames", how="inner"
)
found_by_name["nif"] = (
    found_by_name[["nif_x", "nif_y"]]
    .applymap(validate_nif)
    .apply(lambda x: list(set([el for el in x if el])), axis=1)
)
found_by_name = (
    found_by_name.groupby("ID_EMPRESA")
    .agg(
        {
            "nif": sum,
            "index": set,
            "UsedNames": set,
            "NameProposed": set,
            "isPYME": set,
            "UTE": set,
        }
    )
    .applymap(lambda x: list(set(x)))
    .reset_index()
)
found_by_name

In [ ]:
# Unimos los dos dataframes
found = (
    pd.concat([found_by_nif, found_by_name])
    .groupby("ID_EMPRESA")
    .agg(sum)
    .applymap(lambda x: list(set(x)) if x else None)
    .reset_index()
)
not_found = emp_zgz[~emp_zgz["ID_EMPRESA"].isin(found["ID_EMPRESA"])]
# Stats
print(f"Se han encontrado {len(found)} empresas de Zaragoza en las empresas previas")
print("Elementos inválidos en los datos:")
print(found.apply(pd.isna, axis=1).sum().to_dict())
print(f"Hay {len(not_found)} empresas que no se han encontrado")
# found[["nif", "index", "UsedNames", "NameProposed", "isPYME", "UTE"]].applymap(
#     lambda x: x[0]
# )

## Análisis empresas encontradas y no encontradas

### Incorrect data

In [ ]:
null_id_emp_ids = emp_zgz[emp_zgz["nif"].isna()]["ID_EMPRESA"].values

print(f"Null ids: {len(null_id_emp_ids)}")
print(f"Null ids found: {found['ID_EMPRESA'].isin(null_id_emp_ids).sum()}")
print(f"Null ids not found: {not_found['ID_EMPRESA'].isin(null_id_emp_ids).sum()}")

In [ ]:
print(f"De las empresas que no se han encontrado ({len(not_found)})")
not_found_valid = not_found[not_found["valid_nif"].apply(bool)]
not_found_invalid = not_found[~not_found["valid_nif"].apply(bool)].copy()
print(
    f"Hay {len(not_found_valid)} que tienen un NIF válido y {len(not_found_invalid)} que no"
)

# Intento de corrección del nif
not_found_invalid["proposed_nif"] = (
    not_found_invalid["nif"].dropna().apply(validate_nif, correct=True)
)
print(f"De los no encontrados, que tienen NIF no nulo, se han podido corregir:")
not_found_invalid.dropna(subset="proposed_nif")

In [ ]:
# Columnas que deberían tener valores únicos
unique_columns = [c for c in found.columns if c not in ["ID_EMPRESA", "UsedNames"]]
found[unique_columns] = found[unique_columns].applymap(
    lambda x: [el for el in x if not pd.isna(el)]
)

In [ ]:
# Encontradas
unique = (
    found[unique_columns]
    .applymap(lambda x: len(x) if x else 1)
    .apply(lambda x: all(x == 1), axis=1)
)
# Válidas (todo son valores únicos)
valid_unique = found[unique].copy()
valid_unique[unique_columns] = valid_unique[unique_columns].applymap(
    lambda x: x[0] if x else None
)

# Inválidas (algún valor que debería ser único no lo es)
invalid_unique = found[~unique].copy()
# invalid_unique[unique_columns] = invalid_unique[unique_columns].applymap(lambda x: x if x else None)

In [ ]:
# # TODO: Para Zaragoza, podemos sugerir un NIF en función de si la provincia es Zaragoza:
# invalid_unique["nif"].apply(
#     lambda x: [get_info_from_cif(el)[0] == "Zaragoza" for el in x]
# )

#### UTEs

In [ ]:
emp_zgz.loc[emp_zgz["UTE"] == True, "empresa"].values

In [ ]:
print(valid_unique["UTE"].value_counts().to_dict())
print(invalid_unique["UTE"].value_counts())

In [ ]:
found["UTE"]

In [ ]:
valid_unique[valid_unique["UTE"] == True]

In [ ]:
# Empresas no encontradas en datos propios, con NIF válido
not_found = emp_zgz.loc[
    ~emp_zgz["ID_EMPRESA"].isin(found["ID_EMPRESA"]) & emp_zgz["valid_nif"].apply(bool)
]

In [ ]:
# # Rellenar NIF:
# empty_nif = emp_zgz[
#     (emp_zgz["ID_EMPRESA"].isin(found["ID_EMPRESA"])) & (emp_zgz["nif"].isna())
# ]
# pd.merge(
#     empty_nif[["ID_EMPRESA", "empresa", "empresa_proc"]],
#     found,
#     left_on="ID_EMPRESA",
#     right_on="ID_EMPRESA",
#     how="inner",
# )

## OLD

In [ ]:
# # df = pd.read_csv("data/empresas.csv", sep=";", header=0, nrows=64, index_col=False)
# # df = pd.read_excel("data/empresas.xlsx")

# with open("data/empresas_zgz.csv", "r", encoding="utf-8") as f:
#     emp = [
#         [el.replace('"', "").strip() for el in l.lower().strip().split(";", 4)]
#         for l in f.readlines()
#         if len(l) > 2
#     ]
# cols = emp[0]
# data = emp[1:]
# emp_zgz = pd.DataFrame(data=data, columns=cols)
# emp_zgz = emp_zgz.applymap(lambda x: x if x else None)
# emp_zgz = emp_zgz.dropna(how="all").drop_duplicates().reset_index(drop=True)
# emp_zgz["empresa"] = emp_zgz["empresa"].apply(clean_company_type)
# emp_zgz["empresa_proc"] = emp_zgz["empresa"].apply(clean_company_type, remove_type=True)
# emp_zgz["nif"] = emp_zgz["nif"].apply(lambda x: regex.sub(r"\W", "", x) if x else None)
# emp_zgz["nif_type"] = emp_zgz["nif"].apply(lambda x: get_nif_type(x) if x else None)